In [5]:
from collections import deque

class Graph:
    def __init__(self, adjacency_list):
        self.adjacency_list = adjacency_list

    def get_neighbors(self, v):
        return self.adjacency_list[v]

    def h(self, n):
        H = {
            'A': 4,
            'B': 2,
            'C': 6,
            'D': 8,
            'E': 6,
            'F': 4,
            'G': 0
        }
        return H[n]

    def a_star_algorithm(self, start_node, stop_node):
        open_list = set([start_node])
        closed_list = set([])

        g = {}
        g[start_node] = 0

        parents = {}
        parents[start_node] = start_node

        while len(open_list) > 0:
            n = None
            for v in open_list:
                if n is None or g[v] + self.h(v) < g[n] + self.h(n):
                    n = v

            if n is None:
                print('Path does not exist!')
                return None

            if n == stop_node:
                reconst_path = []
                while parents[n] != n:
                    reconst_path.append(n)
                    n = parents[n]
                reconst_path.append(start_node)
                reconst_path.reverse()
                print('Path found: {}'.format(reconst_path))
                return reconst_path

            for (m, weight) in self.get_neighbors(n):
                if m not in open_list and m not in closed_list:
                    open_list.add(m)
                    parents[m] = n
                    g[m] = g[n] + weight
                else:
                    if g[m] > g[n] + weight:
                        g[m] = g[n] + weight
                        parents[m] = n
                        if m in closed_list:
                            closed_list.remove(m)
                            open_list.add(m)

            open_list.remove(n)
            closed_list.add(n)

        print('Path does not exist!')
        return None

# Updated adjacency list with valid nodes only
adjacency_list = {
    'A': [('B', 2), ('C', 1)],
    'B': [('D', 5), ('E', 3)],
    'C': [('F', 4)],
    'D': [],
    'E': [('G', 2)],
    'F': [('G', 1)],
    'G': []
}

graph1 = Graph(adjacency_list)
graph1.a_star_algorithm('A', 'G')  # ✅ Updated 'S' to 'A'


Path found: ['A', 'C', 'F', 'G']


['A', 'C', 'F', 'G']

In [9]:
from copy import deepcopy
import numpy as np
import time

def bestsolution(state):
    path = []
    idx = len(state) - 1
    while idx != -1:
        path.insert(0, state[idx]['puzzle'])
        idx = state[idx]['parent']
    return np.array(path).reshape(-1, 3, 3)

def misplaced_tiles(puzzle, goal):
    return sum(1 for i in range(9) if puzzle[i] != goal[i] and puzzle[i] != 0)

def get_neighbors(index):
    moves = []
    row, col = divmod(index, 3)
    if row > 0: moves.append(index - 3)
    if row < 2: moves.append(index + 3)
    if col > 0: moves.append(index - 1)
    if col < 2: moves.append(index + 1)
    return moves

def a_star_8_puzzle(start, goal):
    start_time = time.time()

    state = [{
        'puzzle': start,
        'parent': -1,
        'g': 0,
        'h': misplaced_tiles(start, goal)
    }]
    open_list = [(0, 0)]  # (index in state, f = g + h)
    visited = []

    while open_list:
        open_list.sort(key=lambda x: x[1])  # sort by f(n)
        current_index, _ = open_list.pop(0)
        current = state[current_index]
        visited.append(tuple(current['puzzle']))

        if current['puzzle'] == goal:
            print("✅ Puzzle is solvable.")
            return state, len(visited)

        blank = current['puzzle'].index(0)
        for move in get_neighbors(blank):
            new_puzzle = current['puzzle'][:]
            new_puzzle[blank], new_puzzle[move] = new_puzzle[move], new_puzzle[blank]

            if tuple(new_puzzle) in visited:
                continue

            g = current['g'] + 1
            h = misplaced_tiles(new_puzzle, goal)
            state.append({
                'puzzle': new_puzzle,
                'parent': current_index,
                'g': g,
                'h': h
            })
            open_list.append((len(state) - 1, g + h))

        # Optional: timeout check
        if time.time() - start_time > 10:
            print("❌ Timeout! Puzzle may not be solvable.")
            return state, len(visited)

    print("❌ Puzzle is not solvable.")
    return state, len(visited)

# Test it with sample input
start = [2, 8, 3, 1, 6, 4, 7, 0, 5]
goal = [1, 2, 3, 8, 0, 4, 7, 6, 5]

state, visited = a_star_8_puzzle(start, goal)
path = bestsolution(state)

# Print the solution path
for step in path:
    print(step, "\n")

print("Total steps to reach goal:", len(path) - 1)
print("Total nodes visited:", visited)


✅ Puzzle is solvable.
[[2 8 3]
 [1 6 4]
 [7 0 5]] 

[[2 8 3]
 [1 0 4]
 [7 6 5]] 

[[2 0 3]
 [1 8 4]
 [7 6 5]] 

[[0 2 3]
 [1 8 4]
 [7 6 5]] 

[[1 2 3]
 [0 8 4]
 [7 6 5]] 

[[1 2 3]
 [8 0 4]
 [7 6 5]] 

Total steps to reach goal: 5
Total nodes visited: 7
